In [141]:
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import math

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import f1_score


In [73]:
DATA_DIR = pathlib.Path("Z:\\Dropbox\\www\\steelfeet.ru\\_hack\\2021-22\global-ai")
#DATA_DIR = pathlib.Path("C:\\0")
DATA_FILE = "train.csv"
TEST_FILE = "test.csv"
#group_by столбцы
AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]


# Загрузка данных

In [32]:
data =[]
max_string = 0;
with open(DATA_DIR.joinpath(DATA_FILE), 'r') as file_name:
    reader = csv.DictReader(file_name)
    for row in reader:
        data.append((row["Smiles"], row["Active"]))
        if (len(row["Smiles"]) > max_string):
            max_string = len(row["Smiles"])




print(len(data))

5557


# Считаем статистику символов

In [125]:
summ_data = {}
freq_data = {}
for row in data[:5000]:
    smile, active = row
    n=0
    for simbol in smile:
        s1 = str(smile[n])
        try:
            s12 = str(smile[n]+smile[n+1])
        except:
            s12 = ""
        try:
            s21 = str(smile[n-1]+smile[n])
        except:
            s21 = ""




        if ((active=="True") and (len(s12) > 0) and (len(s21) > 0)):
            try:
                summ_data[s1] = summ_data[s1] + 1
                summ_data[s12] = summ_data[s12] + 1
                summ_data[s21] = summ_data[s21] + 1
            except:
                summ_data[s1] = 1
                summ_data[s12] = 1
                summ_data[s21] = 1

        try:
            freq_data[s1] = freq_data[s1] + 1
            freq_data[s12] = freq_data[s12] + 1
            freq_data[s21] = freq_data[s21] + 1
        except:
            freq_data[s1] = 1
            freq_data[s12] = 1
            freq_data[s21] = 1

        n+=1

stat_data = {}
clear_stat_data = {}

for simbol in summ_data:
    stat_data[simbol] = summ_data[simbol] / freq_data[simbol]
    if (stat_data[simbol] > 0.2):
        clear_stat_data[simbol] = stat_data[simbol]
    #print(simbol, summ_data[simbol], freq_data[simbol], stat_data[simbol])

print(clear_stat_data)
print(len(clear_stat_data))

{'(O': 0.2603728202044498, 'H': 0.417910447761194, 'l': 0.96875, '2s': 0.6470588235294118, 's': 0.20689655172413793, '[N': 0.28205128205128205, '][': 0.23076923076923078, 'S.': 0.6666666666666666, 'cs': 0.25925925925925924, '5': 0.5555555555555556, ']3': 0.3684210526315789, 'S=': 0.2222222222222222, 'S)': 0.21710526315789475, ')S': 0.20754716981132076, 'SS': 0.2037037037037037, 'rO': 0.5, 'S3': 0.25, '-1': 0.25, '[S': 0.3333333333333333, 'S+': 0.23255813953488372, 'S@': 0.42857142857142855, 'N.': 0.2727272727272727, '.S': 1.0, 'C5': 1.0, 'o+': 0.3333333333333333, '3.': 0.4, 'IO': 0.3333333333333333, '/n': 1.0, '7C': 0.25, 'c8': 1.0, '8': 0.3333333333333333, '8c': 0.5, '86': 0.5, '7)': 0.25, '5N': 0.5, '3I': 1.0, 'Ag': 1.0, 'g': 1.0, 'g+': 1.0, '[s': 1.0, 'se': 0.5, 'e': 0.3333333333333333, 'e]': 0.3333333333333333, 'S-': 1.0, 'o[': 1.0}
45


In [145]:
y_true = []
y_pred = []

for row in data[5001:]:
    smile, active = row
    #print(row)
    bayes = 0.5
    n = 0
    for simbol in smile:
        s1 = str(smile[n])
        try:
            s12 = str(smile[n]+smile[n+1])
        except:
            s12 = ""
        try:
            s21 = str(smile[n-1]+smile[n])
        except:
            s21 = ""

        #$dish_weight = $dish_weight * $this_weight / ($dish_weight*$this_weight + (1-$dish_weight)*(1-$this_weight));
        try:
            bayes = bayes*clear_stat_data[s1] / (bayes*clear_stat_data[s1] + (1-bayes)*(1-clear_stat_data[s1]))
        except:
            pass
        #print(s1, stat_data[s1], bayes)

        try:
            bayes = bayes*clear_stat_data[s12] / (bayes*clear_stat_data[s12] + (1-bayes)*(1-clear_stat_data[s12]))
            #print(s12, stat_data[s12], bayes)
        except:
            pass

        try:
            bayes = bayes*clear_stat_data[s21] / (bayes*clear_stat_data[s21] + (1-bayes)*(1-clear_stat_data[s21]))
        except:
            pass

        n+=1
        print           
    
    if(active=="True"):
        y_true.append(1)
        print(bayes) #1.00e-50
    else:
        y_true.append(0)
    
    
    if (bayes>1.00e-3):
        y_pred.append(1)
    else:
        y_pred.append(0)

print("-------------------- STAT ------------------")
print(f1_score(y_true, y_pred))



0.038127498670657976
0.5
4.4989364837926775e-05
0.13370165745856352
1.0
0.0005670624482446602
0.11026241642353601
0.417910447761194
0.022441651705565512
1.0
0.00023580871778717134
1.0611864002395638e-08
0.5
1.0
0.20689655172413793
0.417910447761194
0.5
0.5
-------------------- STAT ------------------
0.05156537753222837


In [75]:
test =[]
with open(DATA_DIR.joinpath(TEST_FILE), 'r') as file_name:
    reader = csv.DictReader(file_name)
    for row in reader:
        test.append((row["Smiles"]))


print(len(test))

1614


In [14]:
CAT_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month"]
FTS_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month", "vol_tm6", "vol_tm5", "vol_tm4", "vol_tm3", "vol_tm2", "vol_tm1","last_year_avg", "last_year_min", "last_year_max", "test_target"]
TARGET = "target"

In [31]:
model = CatBoostRegressor(iterations=100,
                          early_stopping_rounds=30,
                          depth=4,
                          cat_features=CAT_COLS,
                          random_state=RS,
                          verbose=10)
model.fit(full_features["tr"][FTS_COLS], full_features["tr"][TARGET],
          eval_set=(full_features["val"][FTS_COLS], full_features["val"][TARGET]),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.303515
0:	learn: 235.8068500	test: 245.0794042	best: 245.0794042 (0)	total: 10.7ms	remaining: 1.05s
10:	learn: 35.2997723	test: 40.5404187	best: 40.5404187 (10)	total: 94.9ms	remaining: 768ms
20:	learn: 23.6761191	test: 34.4347514	best: 34.4347514 (20)	total: 171ms	remaining: 641ms
30:	learn: 18.7958621	test: 31.2704405	best: 31.1514443 (29)	total: 247ms	remaining: 550ms
40:	learn: 15.7582627	test: 29.2866352	best: 29.0267818 (39)	total: 325ms	remaining: 467ms
50:	learn: 14.1432641	test: 28.5346731	best: 28.5346731 (50)	total: 394ms	remaining: 379ms
60:	learn: 12.1707141	test: 27.1328503	best: 27.1328503 (60)	total: 464ms	remaining: 297ms
70:	learn: 11.1094871	test: 27.2937926	best: 27.1328503 (60)	total: 542ms	remaining: 221ms
80:	learn: 10.0454705	test: 27.2059462	best: 27.1328503 (60)	total: 613ms	remaining: 144ms
90:	learn: 8.9381386	test: 26.9386519	best: 26.9386519 (90)	total: 686ms	remaining: 67.9ms
99:	learn: 8.2488920	test: 26.9362442	best: 26.7321726 (9

In [32]:
pd.Series(model.feature_importances_, index=FTS_COLS).sort_values(ascending=False)

test_target      92.941078
last_year_avg     2.236702
vol_tm3           2.184160
last_year_max     1.715369
vol_tm6           0.324613
vol_tm2           0.315761
vol_tm1           0.110409
vol_tm4           0.092757
vol_tm5           0.031920
last_year_min     0.021033
region            0.009288
month             0.007578
material_code     0.005026
manager_code      0.004305
company_code      0.000000
country           0.000000
dtype: float64